In [1]:
from sailthru.sailthru_client import SailthruClient
import vertica_python
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from vertica_python import connect
import StringIO
from StringIO import StringIO
from datetime import date, timedelta as td
import xlsxwriter
import os
import json
import urllib2
import base64
import datetime
import re
import csv
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from IPython.display import display
import requests
import argparse
from dateutil import parser
from datetime import datetime, timedelta
from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [2]:
def get_service(api_name, api_version, scope, client_secrets_path):
  """Get a service that communicates to a Google API.

  Args:
    api_name: string The name of the api to connect to.
    api_version: string The api version to connect to.
    scope: A list of strings representing the auth scopes to authorize for the
      connection.
    client_secrets_path: string A path to a valid client secrets file.

  Returns:
    A service that is connected to the specified API.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      client_secrets_path, scope=scope,
      message=tools.message_if_missing(client_secrets_path))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage(api_name + '.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  service = build(api_name, api_version, http=http)

  return service

In [3]:
def get_ga_data(service, config_dict):
    
    df = pd.DataFrame()
    
    for i in pd.date_range(start_date, end_date):
        
        out = service.data().ga().get(
            ids='ga:' + '86300562',
            start_date=str(i.date()),
            end_date=str(i.date()),
            max_results=10000,
            metrics=','.join(config_dict['metric_list']),
            dimensions=','.join(config_dict['dimension_list']),
            filters=';'.join(config_dict['filter_list'])).execute()

        col_list = []

        for i in np.arange(0,len(out['columnHeaders'])):
            line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
            col_list.append(line)
        try:
            df = df.append(pd.DataFrame(out['rows'],columns=col_list))
        except:
            len(out) == 0
            
    return df


In [4]:
def extract_and_clean_df(config_dict, metric):
    
    df = get_ga_data(service, config_dict)
    
    if (metric == 'sessions'):
        df[metric] = pd.to_numeric(df[metric])
    elif (metric != 'transactions'):
        df = df.rename(columns={'uniqueEvents': metric})
        df[metric] = pd.to_numeric(df[metric])
    else:
        df = df.rename(columns={'itemRevenue':'total_revenue',
                                'uniquePurchases':'transactions',
                                'productName':'eventLabel'})
        df['total_revenue'] = pd.to_numeric(df['total_revenue'])
        df['transactions'] = pd.to_numeric(df['transactions'])
    return df

In [5]:
def massage_df(df, metric):
    
    max_df = df.groupby('campaign').max()
    
    if (metric != 'transactions'):
        df_agg = df.reset_index().merge(df.groupby('campaign').sum().reset_index(), on='campaign',how='inner')
        df_agg = df_agg.rename(columns={'eventLabel_x':'top_%s_course' % metric,
                                        '%s_x' % metric:'%s_in_top_course' % metric,
                                        '%s_y' % metric:'%s_from_campaign' % metric                                                     
                                       })
        df_agg = df_agg.drop(['eventLabel_y'],axis=1)
    else:
        df_agg = df.merge(max_df.reset_index(), how='inner', on=['campaign', 'total_revenue', 'transactions']).groupby('campaign').first()
        df_agg = df_agg.reset_index().merge(df.groupby('campaign').sum().reset_index(), on='campaign',how='inner')
        df_agg = df_agg.rename(columns={'productName_x':'top_%s_course' % metric,
                                'total_revenue_x':'total_revenue_from_top_course',  
                                '%s_x' % metric:'%s_in_top_course' % metric,
                                'total_revenue_y':'total_revenue_from_campaign',  
                                '%s_y' % metric:'%s_from_campaign' % metric                                                     
                               })
        df_agg = df_agg.drop(['productName_y'],axis=1)
    return df_agg

In [6]:
def extract_payload(row):
    
    try:
        emails_sent = row['stats']['total']['count']
    except:
        emails_sent = 0
    
    try:
        unsubscribed = row['stats']['total']['optout']
    except:
        unsubscribed = 0
        
    try:
        unique_opens = row['stats']['total']['open_total']
    except:
        unique_opens = 0
        
    try:
        unique_sub_clicks = row['stats']['total']['click_total']
    except:
        unique_sub_clicks = 0
        
#     dash_list = [pos for pos, char in enumerate(row['name']) if char == '_']
    
#     if (len(dash_list) > 1):
#         row['name'] = row['name'][0:dash_list[0]] + row['name'][dash_list[1]::]

    return row.append(pd.Series([emails_sent, unsubscribed, unique_opens, unique_sub_clicks]))

In [7]:
def clean_sailthru(df):
    df['campaign'] = df.campaign.str.lower()
    df['campaign'] = df.campaign.str.replace('_','-')
    df = df[df.campaign.str.contains('studentnewsletter')]
    df['unsub_rate'] = df['unsubscribed']*100.0/df['emails_sent']
    df['open_rate'] = df['unique_opens']*100.0/df['emails_sent']
    df['click_rate'] = df['unique_sub_clicks']*100.0/df['emails_sent']
    return df

In [8]:
def ga_timestamp(row):
    
    if (str(row['course_id'])=='nan'):
        row['date_diff'] = 0
    else:
        row['date'] = parser.parse(row['date'])
        row['date_diff'] = (row['date'] - row['6d_campaign_end']).days
    return row

def sailthru_timestamp(row, date_col='start_time'):
    row[date_col] = parser.parse(row[date_col])
    return row

In [9]:
def add_metrics(df):
    df['registrations_conversion_rate'] = df['registrations']*100.0/df['emails_sent']
    df['enrollments_conversion_rate'] = df['enrollments']*100.0/df['emails_sent']
    df['dollars_per_email'] = df['total_revenue']*100.0/df['emails_sent']
    return df

In [10]:
scope = ['https://www.googleapis.com/auth/analytics.readonly']
service = get_service('analytics', 'v3', scope, 'client_secrets.json')

In [11]:
start_date = '2016-01-01'
end_date = '2017-02-01'

In [12]:
sessions_dict = {
    'filter_list': [
'ga:campaign=~newsletter',
'ga:medium==email',
'ga:source==sailthru'

],
    'dimension_list': [
'ga:date',
'ga:campaign',
],
    'metric_list': [
'ga:sessions',
],
}

In [13]:
enrollments_dict = {
    'filter_list': [
'ga:eventAction==edx.course.enrollment.activated',
'ga:campaign=~newsletter',
'ga:medium==email',
'ga:source==sailthru'

],
    'dimension_list': [
'ga:date',
'ga:campaign',
'ga:eventLabel'
],
    'metric_list': [
'ga:uniqueEvents'
],
}

In [14]:
registrations_dict = {
    'filter_list': [
'ga:eventAction==edx.bi.user.account.registered',
'ga:campaign=~newsletter',
'ga:medium==email',
'ga:source==sailthru'

],
    'dimension_list': [
'ga:date',
'ga:campaign',
'ga:eventLabel'
],
    'metric_list': [
'ga:uniqueEvents'
],
}

In [110]:
transactions_dict = {
    'filter_list': [
'ga:campaign=~newsletter',
'ga:medium==email',
'ga:source==sailthru'

],
    'dimension_list': [
'ga:date',
'ga:campaign',
'ga:productName',
],
    'metric_list': [
'ga:itemRevenue',
'ga:uniquePurchases'
],
}

In [ ]:
ga_sessions = extract_and_clean_df(sessions_dict, 'sessions')
ga_registrations = extract_and_clean_df(registrations_dict, 'registrations')
ga_enrollments = extract_and_clean_df(enrollments_dict, 'enrollments')
ga_transactions = extract_and_clean_df(transactions_dict, 'transactions')

In [ ]:
print 'jobs done'

In [52]:
#combine all ga data into one df

ga_master = (ga_registrations.merge(
        ga_enrollments, how='outer', on = ['campaign','eventLabel','date'])).merge(
    ga_transactions, how='outer', on = ['campaign','eventLabel','date']).merge(
    ga_sessions, how='inner', on = ['campaign', 'date']).reset_index()
ga_master = ga_master.drop(['index'],axis=1)
ga_master = ga_master.rename(columns={'eventLabel':'course_id'})
ga_master['campaign'] = ga_master.campaign.str.lower()
ga_master['campaign'] = ga_master.campaign.str.replace('_','-')

In [12]:
api_key = 'd50d1dac0ae6f50b885a618a7d36d4e0'
api_secret = '844bf4965e85e43da2267b8ac0fb5c1c'
sc = sailthru_client = SailthruClient(api_key, api_secret)

request_data = {'status': 'sent', 'start_date':'2017-01-01'}
response = sc.api_get('blast', request_data).get_body()

request_data_user = {'status': 'sent', 'start_date':'2017-01-01'}


#sailthru_campaign_stats = DataFrame(response['blasts'])[['name','blast_id','start_time','stats']]

In [14]:
sc.api_get('blast', {'status': 'sent'}).get_body()

{'blast_count': 624,
 'blasts': [{'blast_id': 9321641,
   'copy_blast_id': 8825408,
   'data_feed_url': 'http://feed.sailthru.com/ws/feed?id=5877aa2d1aa312757b8b457e',
   'email_count': 974151,
   'labels': ['Themed NL'],
   'list': 'Interested in Social Science',
   'mode': 'email',
   'modify_time': 'Wed, 05 Apr 2017 16:43:16 -0400',
   'modify_user': 'jzheng@edx.org',
   'name': 'CivicsNewsletter_20170406',
   'public_url': 'link.edx.org/public/9321641',
   'schedule_time': 'Thu, 06 Apr 2017 10:00:00 -0400',
   'sent_count': 634577,
   'start_time': 'Thu, 06 Apr 2017 10:00:03 -0400',
   'stats': {'total': {'beacon_click': 4364,
     'beacon_noclick': 60157,
     'click_multiple_urls': 1358,
     'click_total': 7496,
     'count': 634577,
     'hardbounce': 56,
     'nobeacon_click': 357,
     'open_total': 79939,
     'optout': 415,
     'purchase': 1441,
     'purchase_first': 24,
     'purchase_price': 244200,
     'purchase_second': 107,
     'pv': 8879,
     'rev': 244200,
     

In [21]:
sc.api_get('list', {'list': 'Disengaged Learners'}).get_body()

{'count': 1377887,
 'create_time': 'Wed, 12 Oct 2016 15:03:55 -0400',
 'description': "Members of list 'All edX Users': engagement level is disengaged, and 'activated' is 1",
 'list': 'Disengaged Learners',
 'list_id': '57fe891b6e4adcd07f8b471f',
 'primary': False,
 'public_name': 'Disengaged Learners',
 'send_time': 'Tue, 04 Apr 2017 14:36:02 -0400',
 'type': 'smart'}

In [27]:
sc.api_post('job', {'job': 'blast_query', 'blast_id': '9259889'}).get_body()

{'job_id': '58e6a657ff530acd228b4c4b',
 'name': 'Export Mass Mail Data',
 'status': 'pending'}

In [26]:
sc.api_get('job', {'job_id':'58e687756e4adcec458b5554'}).get_body()

{'blast_id': 9259889,
 'end_time': 'Thu, 06 Apr 2017 14:34:32 -0400',
 'export_url': 'https://s3.amazonaws.com/sailthru/export/2017/04/06/7005a4c5ad7d1c7113809d2a91b05307',
 'filename': 'export-blast-9259889.csv',
 'job_id': '58e687756e4adcec458b5554',
 'name': 'Export Mass Mail Data',
 'start_time': 'Thu, 06 Apr 2017 14:23:03 -0400',
 'status': 'completed'}

In [89]:
sc.api_get('job', {'job': 'blast_query'}).get_body()

{'error': 2, 'errormsg': 'Missing required parameter: job_id'}

In [93]:
sailthru_campaign_stats = DataFrame(response['blasts'])[['name','blast_id','start_time','abtest','abtest_segment','list']]

In [94]:
sailthru_campaign_stats

,name,blast_id,start_time,abtest,abtest_segment,list
0,StudentNewsletter_Inactives_20170328,9259477,"Tue, 28 Mar 2017 14:06:04 -0400",final,Final,Disengaged Learners
1,StudentNewsletter_Actives_20170328,9254400,"Tue, 28 Mar 2017 14:05:01 -0400",final,Final,NL Actives
2,edX Business Outreach - Part 2,9218942,"Tue, 28 Mar 2017 13:00:04 -0400",NaN,NaN,Business Development
3,StudentNewsletter_Inactives_20170328,9259892,"Tue, 28 Mar 2017 11:48:03 -0400",final,D,Disengaged Learners
4,StudentNewsletter_Inactives_20170328,9259889,"Tue, 28 Mar 2017 11:48:03 -0400",final,C,Disengaged Learners
5,StudentNewsletter_Inactives_20170328,9259479,"Tue, 28 Mar 2017 11:48:03 -0400",final,B,Disengaged Learners
6,StudentNewsletter_Inactives_20170328,9259478,"Tue, 28 Mar 2017 11:48:03 -0400",final,A,Disengaged Learners
7,StudentNewsletter_Inactives_20170328,9258898,"Tue, 28 Mar 2017 10:29:02 -0400",final,D,Disengaged Learners
8,StudentNewsletter_Inactives_20170328,9258897,"Tue, 28 Mar 2017 10:29:02 -0400",final,C,Disengaged Learners
9,StudentNewsletter_Inactives_20170328,9258883,"Tue, 28 Mar 2017 10:29:02 -0400",final,B,Disengaged Learners


In [11]:
#extract data, rename columns

sailthru_campaign_stats = sailthru_campaign_stats.apply(extract_payload, axis=1)
sailthru_campaign_stats = sailthru_campaign_stats.rename(columns={'name':'campaign',
                    0:'emails_sent',
                   1:'unsubscribed',
                   2:'unique_opens',
                   3:'unique_sub_clicks'})
sailthru_campaign_stats = sailthru_campaign_stats.apply(sailthru_timestamp, axis=1)
sailthru_campaign_stats['6d_campaign_end'] = sailthru_campaign_stats.start_time + timedelta(days=6) 

In [92]:
#de-dupe table to map one sailthru campaign to a 6d end date

sailthru_campaign_6d = sailthru_campaign_stats.groupby('campaign').first()['6d_campaign_end'].reset_index()
sailthru_campaign_6d['campaign'] = sailthru_campaign_6d.campaign.str.lower().str.replace('_','-')
sailthru_campaign_6d = sailthru_campaign_6d[sailthru_campaign_6d.campaign.str.contains('studentnewsletter')]

In [94]:
sailthru_campaign_stats_summarized = (sailthru_campaign_stats.groupby('campaign').sum()).reset_index().drop(['blast_id'], axis=1)
sailthru_campaign_stats_summarized = clean_sailthru(sailthru_campaign_stats_summarized)
sailthru_campaign_stats_summarized = sailthru_campaign_stats_summarized.merge(sailthru_campaign_6d, how='inner', on='campaign')

/Users/adleyhemphill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/adleyhemphill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/adleyhemphill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [100]:
def df_agg(df, agg_level):
    
    if (agg_level == 'course'):
        df_all = df.groupby(['campaign','course_id']).sum().reset_index()
        df_6d = (df[df.date_diff<=6].groupby(['campaign','course_id']).sum().reset_index())
        df_out = df_all.merge(df_6d, how='left', on=['campaign','course_id'], suffixes=('','_6d'))
        return df_out
    elif (agg_level == 'campaign'):
        df_all = df.groupby(['campaign']).sum().reset_index()
        df_6d = (df[df.date_diff<=7].groupby(['campaign']).sum().reset_index())
        df_out =  df_all.merge(df_6d, how='left', on=['campaign'], suffixes=('','_6d'))
        return df_out

In [101]:
#combine all ga and sailthru data and clean up the date info so we can do the 7d calculations

ga_sailthru_combined = sailthru_campaign_stats_summarized[['campaign','6d_campaign_end']].merge(ga_master, how='left', on='campaign')
ga_sailthru_combined = ga_sailthru_combined.apply(ga_timestamp, axis=1)

In [102]:
#aggregate at the campaign and course level

ga_sailthru_combined_campaign = df_agg(ga_sailthru_combined, 'campaign')
ga_sailthru_combined_course = df_agg(ga_sailthru_combined, 'course')

In [103]:
#join back to get the sailthru stats

ga_sailthru_combined_campaign = sailthru_campaign_stats_summarized.merge(
    ga_sailthru_combined_campaign, how='left', on='campaign').drop(['date_diff','date_diff_6d','6d_campaign_end'],axis=1)
ga_sailthru_combined_course = sailthru_campaign_stats_summarized.merge(
    ga_sailthru_combined_course, how='left', on='campaign').drop(['date_diff','date_diff_6d','6d_campaign_end'],axis=1)

In [104]:
#add conversion rate metrics

ga_sailthru_combined_campaign = add_metrics(ga_sailthru_combined_campaign)
ga_sailthru_combined_course = add_metrics(ga_sailthru_combined_course)

In [105]:
ga_sailthru_combined_campaign[ga_sailthru_combined_campaign.campaign=='studentnewsletter-actives-20170103']

,campaign,emails_sent,unsubscribed,unique_opens,unique_sub_clicks,unsub_rate,open_rate,click_rate,registrations,enrollments,...,transactions,sessions,registrations_6d,enrollments_6d,total_revenue_6d,transactions_6d,sessions_6d,registrations_conversion_rate,enrollments_conversion_rate,dollars_per_email
45,studentnewsletter-actives-20170103,3116097,7226,1183843,123269,0.231893,37.991211,3.955878,175.0,6991.0,...,81.0,11625637.0,155.0,6613.0,3814.9,67.0,11543856.0,0.005616,0.224351,0.161125


In [ ]:
#master_ga_sailthru_df = master_ga_sailthru_df.replace(to_replace='event', value='homepage')

In [ ]:
excelFile='./../script_output/newsletter_sample2.xlsx'
writer = pd.ExcelWriter(excelFile,engine='xlsxwriter')

ga_sailthru_combined_campaign.to_excel(writer, index=False, sheet_name='campaign_summary', startrow=2)
ga_sailthru_combined_course.to_excel(writer, index=False, sheet_name='course_summary', startrow=2)
ga_sailthru_combined_campaign[ga_sailthru_combined_campaign.campaign.str.contains('^studentnewsletter')].to_excel(writer, index=False, sheet_name='tuesday_campaign_summary', startrow=2)
ga_sailthru_combined_course[ga_sailthru_combined_course.campaign.str.contains('^studentnewsletter')].to_excel(writer, index=False, sheet_name='tuesday_course_summary', startrow=2)

# Get access to the workbook and sheet
workbook = writer.book
worksheet = writer.sheets['campaign_summary']

money_fmt = workbook.add_format({'num_format': '$#,##0', 'bold': True})
money_fmt_long = workbook.add_format({'num_format': '$#,##0.00000', 'bold': True})
percent_fmt = workbook.add_format({'num_format': '0.0000%', 'bold': False})
comma_fmt_long = workbook.add_format({'num_format': '#,##0.000', 'bold': False})
comma_fmt_short = workbook.add_format({'num_format': '#,##0', 'bold': False})
date_fmt = workbook.add_format({'num_format': 'dd/mm/yy'})
cell_format = workbook.add_format({'bold': True, 'italic': False})

worksheet.conditional_format('Q1:Q10000', {'type': '3_color_scale'})
worksheet.conditional_format('R1:R10000', {'type': '3_color_scale'})
worksheet.conditional_format('S1:S10000', {'type': '3_color_scale'})

worksheet.set_column('A:A', 15, date_fmt)
worksheet.set_column('B:H', 15, comma_fmt_long)
worksheet.set_column('I:J', 20, comma_fmt_short)
worksheet.set_column('K:K', 20, money_fmt)
worksheet.set_column('L:N', 20, comma_fmt_short)
worksheet.set_column('O:O', 20, money_fmt)
worksheet.set_column('P:P', 20, comma_fmt_short)
worksheet.set_column('Q:R', 20, comma_fmt_long)
worksheet.set_column('S:S', 20, money_fmt_long)

#worksheet.write('A1', 'Newsletters with over 1M sends, Data from '+str(report_d1)+' to '+str(d2) , cell_format)

worksheet = writer.sheets['course_summary']

worksheet.conditional_format('R1:R10000', {'type': '3_color_scale'})
worksheet.conditional_format('S1:S10000', {'type': '3_color_scale'})
worksheet.conditional_format('T1:T10000', {'type': '3_color_scale'})

worksheet.set_column('A:A', 15, date_fmt)
worksheet.set_column('B:I', 15, comma_fmt_long)
worksheet.set_column('J:K', 20, comma_fmt_short)
worksheet.set_column('L:L', 20, money_fmt)
worksheet.set_column('M:O', 20, comma_fmt_short)
worksheet.set_column('P:P', 20, money_fmt)
worksheet.set_column('Q:Q', 20, comma_fmt_short)
worksheet.set_column('R:S', 20, comma_fmt_long)
worksheet.set_column('T:T', 20, money_fmt_long)


worksheet = writer.sheets['tuesday_campaign_summary']

worksheet.conditional_format('Q1:Q10000', {'type': '3_color_scale'})
worksheet.conditional_format('R1:R10000', {'type': '3_color_scale'})
worksheet.conditional_format('S1:S10000', {'type': '3_color_scale'})

worksheet.set_column('A:A', 15, date_fmt)
worksheet.set_column('B:H', 15, comma_fmt_long)
worksheet.set_column('I:J', 20, comma_fmt_short)
worksheet.set_column('K:K', 20, money_fmt)
worksheet.set_column('L:N', 20, comma_fmt_short)
worksheet.set_column('O:O', 20, money_fmt)
worksheet.set_column('P:P', 20, comma_fmt_short)
worksheet.set_column('Q:R', 20, comma_fmt_long)
worksheet.set_column('S:S', 20, money_fmt_long)

worksheet = writer.sheets['tuesday_course_summary']

worksheet.conditional_format('R1:R10000', {'type': '3_color_scale'})
worksheet.conditional_format('S1:S10000', {'type': '3_color_scale'})
worksheet.conditional_format('T1:T10000', {'type': '3_color_scale'})

worksheet.set_column('A:A', 15, date_fmt)
worksheet.set_column('B:I', 15, comma_fmt_long)
worksheet.set_column('J:K', 20, comma_fmt_short)
worksheet.set_column('L:L', 20, money_fmt)
worksheet.set_column('M:O', 20, comma_fmt_short)
worksheet.set_column('P:P', 20, money_fmt)
worksheet.set_column('Q:Q', 20, comma_fmt_short)
worksheet.set_column('R:S', 20, comma_fmt_long)
worksheet.set_column('T:T', 20, money_fmt_long)

writer.save()